In [1]:
%load_ext autoreload

%autoreload 2

import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib.tri as mtri
from math import sqrt, log
import scipy as sc
from scipy.stats import chi2

from ruspy.data.data_reading import data_reading
from ruspy.data.data_processing import data_processing
from ruspy.estimation.estimation import estimate, estimate_transitions
from ruspy.simulation.robust_sim import draw_trans_probs_mulitvar, calc_cov_multinomial
from ruspy.plotting.prob_simplex import create_set, Kullback_Leibler, get_x

In [2]:
data_reading()

In [63]:
#Initilize a dictionary with values needed on the way.
seed = 122
np.random.seed(seed)
beta = 0.9999
init_dict = {
    'groups': 'group_4',
    'beta': beta,
    'maint_func': 'linear',
    'binsize': 5000,
    'states': 100,
}

In [25]:
repl_data = data_processing(init_dict)

In [26]:
trans_results, cost_results = estimate(init_dict, repl_data)
print(trans_results, cost_results)

{'state_count': array([[68, 94,  9, ...,  0,  0,  0],
       [ 0, 44, 59, ...,  0,  0,  0],
       [ 0,  0, 47, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  1,  1,  0],
       [ 0,  0,  0, ...,  0,  1,  1],
       [ 0,  0,  0, ...,  0,  0,  1]]), 'x': array([0.39189189, 0.59529357, 0.01281454]), 'fun': 3140.5705570938244, 'trans_count': [1682, 2555, 55]}       fun: 163.58247580496428
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-8.52651283e-06,  1.70530257e-05])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 48
      nit: 15
   status: 0
  success: True
        x: array([10.05564506,  2.27695601])


In [27]:
p_ml = trans_results['x']

In [28]:
n= 4292
cov = calc_cov_multinomial(n, p_ml)
np.sqrt(cov[0, 0])

0.0074514995482480514

In [29]:
# Specify simplex
l_simp = sqrt(4 / 3)

In [64]:
probs = draw_trans_probs_mulitvar(n, p_ml, size=100, seed=seed)
draws_y = []
draws_x = []
for p in probs:
    draws_y += [p[1]]
    draws_x += [get_x(p, l_simp)]

In [48]:
roh = chi2.ppf(0.95, len(trans_results['x']) - 1) / (2 * n)
roh

0.0006979804924403517

In [49]:
min_grid_p_0 = min_grid_p_1 = 0
max_grid_p_0 = max_grid_p_1 = 1
step = 0.001
x_set_pre, y_set_pre, set_coord_pre, set_probs_pre = create_set(
    p_ml, roh, l_simp, step, min_grid_p_0, min_grid_p_1, max_grid_p_0, max_grid_p_1, set_method='Kullback')

In [50]:
min_grid_p_0 = set_probs_pre[:, 0].min() - step
max_grid_p_0 = set_probs_pre[:, 0].max() + step
min_grid_p_1 = set_probs_pre[:, 1].min() - step
max_grid_p_1 = set_probs_pre[:, 1].max() + step

In [51]:
step = 0.0001
x_set, y_set, set_coord, set_probs = create_set(
    p_ml, roh, l_simp, step, min_grid_p_0, min_grid_p_1, max_grid_p_0, max_grid_p_1, set_method='Kullback')

In [52]:
hull = sc.spatial.ConvexHull(set_coord)

In [67]:
# Create triangulation.range(
x = np.array([0, 0.5 * l_simp, l_simp])
y = np.array([0, 1, 0])
triang = mtri.Triangulation(x, y)

# Set up thered figure
fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(111)
ax.text(-0.05, -0.03, "Increase by 0")
ax.text(l_simp - 0.06, -0.03, "Increase by 2")
ax.text(0.5 * l_simp - 0.05, 1.02, "Increase by 1")
# ax.plot(x_set, y_set, color='grey')
for simplex in hull.simplices:
    plt.plot(set_coord[simplex, 0], set_coord[simplex, 1], 'k-')
ax.plot([get_x(p_ml, l_simp)], [p_ml[1]], marker='o', markersize=2, color="red")
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
ax.axis("off")

# ax.scatter(draws_x, draws_y, s=10, marker='o', color="green")
ax.triplot(triang)

FigureCanvasNbAgg()

In [65]:
count=0
for p in probs:
    if roh - Kullback_Leibler(p, p_ml) >= 0:
        count+= 1
count

95

In [18]:
# Create triangulation.range(
x = np.array([0, 0.5 * l_simp, l_simp])
y = np.array([0, 1, 0])
triang = mtri.Triangulation(x, y)

# Set up the figure
fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(111)
ax.text(-0.05, -0.03, "Increase by 0")
ax.text(l_simp - 0.06, -0.03, "Increase by 2")
ax.text(0.5 * l_simp - 0.05, 1.02, "Increase by 1")
ax.plot([get_x(p_ml, l_simp)], [p_ml[1]], marker='o', markersize=1, color="red")
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

# Plot the triangulation.
#axs.tricontourf(triang, z)

# ax.set_title('Probability simplex')
# Plot heatmap
# ax2 = ax.twinx()
ax.hist2d(np.array(draws_x), np.array(draws_y), bins=2000, range=[[0, l_simp], [0, 1]])
ax.triplot(triang)

FigureCanvasNbAgg()